# Liquidity Stress Tool

Interactive liquidity stress testing tool used to assess whether a fund can meet redemptions under market stress.

The model operates on aggregated liquidity buckets, mirroring how liquidity risk teams perform funding analysis.


In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

In [2]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

from engine.assets import default_liquidity_profile
from engine.liquidity import run_waterfall, apply_stress
from engine.metrics import liquidity_metrics
from engine.assumptions import ASSUMPTIONS
from engine.scenario_translation import translate_scenario


In [3]:
# Helpers
def fmt_bn(x):
    return f"${x / 1e9:,.2f} bn"

def plot_liquidity_coverage(metrics):
    values = [
        metrics["cash_required"] / 1e9,
        metrics["cash_raised"] / 1e9
    ]
    labels = ["Cash Required", "Cash Raised"]

    plt.figure()
    plt.bar(labels, values)
    plt.ylabel("CAD (bn)")
    plt.title("Liquidity Coverage")
    plt.show()


def plot_waterfall(waterfall):
    df = waterfall[waterfall["cash_used"] > 0]

    if df.empty:
        return

    values = df["cash_used"] / 1e9
    labels = df["bucket"]

    plt.figure()
    plt.barh(labels, values)
    plt.xlabel("Cash Raised (CAD bn)")
    plt.title("Liquidity Waterfall")
    plt.show()


def stress_summary(metrics):
    status = "BREACH" if metrics["breach"] else "PASS"

    print("\n--- Stress Summary ---")
    print(f"Result: {status}")
    print(f"Liquidity Coverage: {metrics['liquidity_coverage']:.2f}")
    print(f"Days to Liquidity: {metrics['days_to_liquidity']}")

In [4]:
# sliders
redemption_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.99,
    step=0.01,
    description="Redemption %",
    readout_format=".0%"
)

equity_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.99,
    step=0.01,
    description="Equity Drawdown",
    readout_format=".0%"
)

# checkboxes
freeze_cash = widgets.Checkbox(value = False, description = "Freeze Cash Liquidity")
freeze_t1 = widgets.Checkbox(value = False, description="Freeze T+1 Liquidity")
freeze_t5 = widgets.Checkbox(value = False, description = "Freeze T+5 Liquidity")
freeze_t30 = widgets.Checkbox(value = False, description = "Freeze T+30 Liquidity")
freeze_t90 = widgets.Checkbox(value = False, description = "Freeze T+90 Liquidity")

# full controls
controls = widgets.VBox([equity_slider, redemption_slider, freeze_cash, freeze_t1, freeze_t5, freeze_t30, freeze_t90])

display(controls)

In [5]:
output = widgets.Output()
display(output)

Output()

In [6]:
def run_stress(_=None):
    with output:
        clear_output()

        # Base portfolio (CAD dollars)
        profile = default_liquidity_profile()

        # Liquidity freezes
        if freeze_t1.value:
            profile.loc[
                profile["bucket"] == "Public Equities [T+1]",
                "available"
            ] = False

        if freeze_t5.value:
            profile.loc[
                profile["bucket"] == "Public Credit [T+5]",
                "available"
            ] = False

        # Scenario definition
        scenario = {
            "equity_drawdown": -equity_slider.value,
            "redemption": redemption_slider.value
        }

        # Translate scenario → valuation + liquidity demand
        stressed_profile, liquidity_demand = translate_scenario(
            profile=profile,
            scenario=scenario,
            assumptions=ASSUMPTIONS
        )

        # Apply valuation stress
        stressed_profile = apply_stress(stressed_profile)

        # Run waterfall
        waterfall, summary = run_waterfall(
            stressed_profile=stressed_profile,
            cash_required=liquidity_demand
        )

        metrics = liquidity_metrics(summary)
        stress_summary(metrics)

        print("Scenario")
        print(scenario)

        print("\nMetrics")
        for k, v in metrics.items():
            if k in ["cash_required", "cash_raised"]:
                print(f"{k}: {fmt_bn(v)}")
            elif isinstance(v, float):
                print(f"{k}: {v:.4f}")
            else:
                print(f"{k}: {v}")

        print(f"\nTotal Fund Value: {fmt_bn(profile['market_value'].sum())}")

        print("\nLiquidity Waterfall")
        waterfall_display = waterfall.copy()
        for col in ["stressed_value", "cash_used", "remaining_value"]:
            waterfall_display[col] = waterfall_display[col].apply(fmt_bn)

        display(waterfall_display)
        plot_liquidity_coverage(metrics)
        plot_waterfall(waterfall)


In [7]:
equity_slider.observe(run_stress, names="value")
redemption_slider.observe(run_stress, names="value")
freeze_cash.observe(run_stress, names="value")
freeze_t1.observe(run_stress, names="value")
freeze_t5.observe(run_stress, names="value")
freeze_t30.observe(run_stress, names="value")
freeze_t90.observe(run_stress, names="value")

run_stress()